<a href="https://colab.research.google.com/github/rudolmat/SAKI-Exercise1/blob/master/flair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My Drive/SAKI/Exercise_2

/content/gdrive/My Drive/SAKI/Exercise_2


In [0]:
import os
os.chdir("/content/gdrive/My Drive/SAKI/Exercise_2") 

In [22]:
!pip install flair

In [0]:
# imports 
from flair.data import Corpus
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from typing import List

# columns of "gold standard" ner annotations and text
columns = {3: 'text', 1: 'ner'}

# folder where training and test data are
data_folder = '/content/gdrive/My Drive/SAKI/Exercise_2'

# 2. what tag do we want to predict?
tag_type = 'ner'

In [23]:
downsample = 1.0 # 1.0 is full data, try a much smaller number like 0.01 to test run the code
# 1. get the corpus
corpus: Corpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns,
                                                              train_file='train_res_bilou.txt',
                                                              test_file='test_res_bilou.txt',
                                                              dev_file=None).downsample(downsample)

print(corpus)

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)


2019-06-08 09:50:42,402 Reading data from /content/gdrive/My Drive/SAKI/Exercise_2
2019-06-08 09:50:42,405 Train: /content/gdrive/My Drive/SAKI/Exercise_2/train_res_bilou.txt
2019-06-08 09:50:42,407 Dev: None
2019-06-08 09:50:42,409 Test: /content/gdrive/My Drive/SAKI/Exercise_2/test_res_bilou.txt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated function (or staticmethod) load_column_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:312: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  train_file, column_format
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:318: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  test_file, column_format


Corpus: 28487 train + 3165 dev + 13050 test sentences
[b'<unk>', b'O', b'B-Degree', b'I-Degree', b'B-Designation', b'I-Designation', b'"B-Companies', b'"I-Companies', b'<START>', b'<STOP>']


In [24]:
# 4. initialize embeddings. Experiment with different embedding types to see what gets the best results
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings,FlairEmbeddings

embedding_types: List[TokenEmbeddings] = [
    WordEmbeddings('glove'),
    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings (needs a LONG time to train :-)
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)




/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [25]:
# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/resume-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

# 8. plot training curves (optional)
#from flair.visual.training_curves import Plotter
#plotter = Plotter()
#plotter.plot_training_curves('resources/taggers/example-ner/loss.tsv')
#plotter.plot_weights('resources/taggers/example-ner/weights.txt')

2019-06-08 09:51:27,860 ----------------------------------------------------------------------------------------------------
2019-06-08 09:51:27,862 Evaluation method: MICRO_F1_SCORE
2019-06-08 09:51:28,429 ----------------------------------------------------------------------------------------------------
2019-06-08 09:51:29,587 epoch 1 - iter 0/891 - loss 18.26095390
2019-06-08 09:51:49,990 epoch 1 - iter 89/891 - loss 1.38068662
2019-06-08 09:52:10,779 epoch 1 - iter 178/891 - loss 0.94589162
2019-06-08 09:52:34,235 epoch 1 - iter 267/891 - loss 0.81468068
2019-06-08 09:52:53,892 epoch 1 - iter 356/891 - loss 0.70975846
2019-06-08 09:53:13,557 epoch 1 - iter 445/891 - loss 0.64924223
2019-06-08 09:53:35,489 epoch 1 - iter 534/891 - loss 0.61052196
2019-06-08 09:53:54,956 epoch 1 - iter 623/891 - loss 0.57770874
2019-06-08 09:54:14,353 epoch 1 - iter 712/891 - loss 0.54459399
2019-06-08 09:54:33,919 epoch 1 - iter 801/891 - loss 0.52175227
2019-06-08 09:54:55,558 epoch 1 - iter 890/8

{'dev_loss_history': [tensor(0.2777, device='cuda:0'),
  tensor(0.2024, device='cuda:0'),
  tensor(0.1776, device='cuda:0'),
  tensor(0.1826, device='cuda:0'),
  tensor(0.1698, device='cuda:0'),
  tensor(0.1593, device='cuda:0'),
  tensor(0.1529, device='cuda:0'),
  tensor(0.1406, device='cuda:0'),
  tensor(0.1393, device='cuda:0'),
  tensor(0.1382, device='cuda:0'),
  tensor(0.1335, device='cuda:0'),
  tensor(0.1275, device='cuda:0'),
  tensor(0.1269, device='cuda:0'),
  tensor(0.1349, device='cuda:0'),
  tensor(0.1234, device='cuda:0'),
  tensor(0.1219, device='cuda:0'),
  tensor(0.1312, device='cuda:0'),
  tensor(0.1273, device='cuda:0'),
  tensor(0.1178, device='cuda:0'),
  tensor(0.1329, device='cuda:0'),
  tensor(0.1176, device='cuda:0'),
  tensor(0.1318, device='cuda:0'),
  tensor(0.1184, device='cuda:0'),
  tensor(0.1214, device='cuda:0'),
  tensor(0.1175, device='cuda:0'),
  tensor(0.1264, device='cuda:0'),
  tensor(0.1174, device='cuda:0'),
  tensor(0.1246, device='cuda:0'),
